In [1]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
bert = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
import pandas as pd
df = pd.read_csv('Reddit_Data.csv')

In [24]:
df['clean_comment'] = df.clean_comment.str.strip()

In [25]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
class RedditDataset(Dataset):
    def __init__(self, comments, labels):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.comments = comments
        self.labels = labels

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = self.tokenizer(self.comments[idx], return_tensors='pt')
        label = self.labels[idx]
        return {
            "input_ids": comment['input_ids'].squeeze(0),
            "attention_mask": comment['attention_mask'].squeeze(0),
            'label': torch.LongTensor([label])
        }

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    labels = torch.cat([item['label'] for item in batch], dim=0)
    
    input_ids = pad_sequence(input_ids, batch_first=True)
    attention_masks = pad_sequence(attention_masks, batch_first=True)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels
    }

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(BertClassifier, self).__init__()
        self.num_classes = num_classes
        # Load the pre-trained BERT model
        self.bert = BertModel.from_pretrained(
            'bert-base-uncased',
            output_hidden_states=True
        )
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(768, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

        # Probes
        self.probe0to3 = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Linear(768, num_classes)
        )
        self.probe4to7 = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Linear(768, num_classes)
        )

    def forward(self, inputs, probing=False):
        out = self.bert(**inputs)
        output = self.dropout(out.pooler_output)
        output = self.linear(output)
        output = self.relu(output)
        output = self.softmax(output)
        probe = dict()
        if probing:
            probeInput = out.hidden_states.clone().detach()
            probe0to3 = self.probe0to3(probeInput[3])
            probe4to7 = self.probe4to7(probeInput[7])
            probe['probe0to3'] = probe0to3
            probe['probe4to7'] = probe4to7
            
        return {
            'logits': output,
            'probes': probe
        }